<a href="https://colab.research.google.com/github/joyuno/lguplus_project/blob/main/%EC%95%8C%EB%9C%B0%ED%8F%B0_%EA%B8%8D%EC%A0%95_%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 검색 키워드 목록
#keywords = ["이야기모바일","마블링","마블프로듀스","헬로모바일","아이즈모바일","유모바일","모빙 알뜰폰","티플러스 알뜰폰","알닷","에르엘 알뜰폰","프리티모바일","슈가모바일","Sugar알뜰폰",
#            "에이모바일","KG모바일", "찬스모바일","핀다이렉트", "이지모바일","스노우맨", "세종텔레콤 알뜰폰","보스텔레콤","보스통신","프리텔레콤",
#            "YL모바일","A모바일", "벨류컴모바일","인스모바일","인스코리아알뜰폰","리브엠","리브모바일","모나 알뜰폰","토스모바일","유플러스","코나아이","EG모바일","이지모바일","코드모바일",
#            "에넥스텔레콤","스마텔","위너스텔레콤","친구모바일","유니컴즈","KPMO","KB알뜰폰","에스원","엔티온텔레콤","LGU+","엘지유플러스",
#            "헬로비전","핀샷알뜰폰","CK모바일","아시아모바일","liv모바일","시월모바일"]

In [ ]:
import os
os.getcwd()

'/content'

In [1]:
import re

# <b> 태그를 제거하는 함수 (문자열만 처리)
def remove_html_tags(text):
    if isinstance(text, str):  # 문자열인 경우만 처리
        return re.sub(r"<.*?>", "", text)
    return text  # 문자열이 아니면 그대로 반환
import requests
import json
from datetime import datetime
import pandas as pd
# 네이버 API 인증 정보
client_id = 'D7rDLLDGwumBgCLpnE2Q'
client_secret = '#'

# 검색 키워드 목록
keywords = ['알뜰폰', '중소 MVNO','알닷']

# 검색 기간 설정
start_date = '2024-10-10'
end_date = datetime.now().strftime('%Y-%m-%d')

# API 호출 함수
def get_news(keyword, start, display):
    url = f"https://openapi.naver.com/v1/search/news.json?query={keyword}&display={display}&start={start}&sort=date"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error Code: {response.status_code}")
        return None

# 각 키워드에 대해 뉴스 기사 수집
all_articles = []
for keyword in keywords:
    start = 1
    display = 40
    while True:
        data = get_news(keyword, start, display)
        if data and 'items' in data:
            for item in data['items']:
                # 기사 작성일자 필터링
                pub_date = datetime.strptime(item['pubDate'], '%a, %d %b %Y %H:%M:%S %z')
                if pub_date.strftime('%Y-%m-%d') < start_date:
                    break
                article = {
                    'title': item['title'],
                    'description': item['description'],
                    'pubDate': pub_date.strftime('%Y-%m-%d %H:%M:%S'),
                    'link': item['link']
                }
                all_articles.append(article)
            start += display
            if start > data['total'] or pub_date.strftime('%Y-%m-%d') < start_date:
                break
        else:
            break

df_news = pd.DataFrame(all_articles)
df_news
# 제목과 본문에서 <b></b> 태그 제거
for item in all_articles:
    item["title"] = remove_html_tags(item.get("title", ""))
    item["description"] = remove_html_tags(item.get("description", ""))



Error Code: 400


In [4]:
df_news.to_csv('/content/drive/MyDrive/sideproject/naver_news/news_MVNO.csv', index=False, encoding='utf-8-sig')

In [2]:
import torch
import transformers
print(torch.__version__)
print(transformers.__version__)

2.6.0+cu124
4.48.3


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
for item in all_articles:
    for key in ["title", "description", "pubDate", "link"]:
        if key not in item or item[key] is None or isinstance(item[key], float):  # NaN(float) 체크
            item[key] = ""  # 빈 문자열로 대체

def filter_similar_articles(articles, similarity_threshold=0.8):
    """
    articles: 뉴스 기사를 담은 리스트 (각 요소는 dict로 'description' 키를 포함)
    similarity_threshold: 코사인 유사도 임계치 (이상일 경우 유사한 기사로 판단)
    """
    # 각 기사의 description을 추출
    descriptions = [article['description'] for article in articles]

    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(descriptions)

    # 코사인 유사도 행렬 계산
    sim_matrix = cosine_similarity(tfidf_matrix)

    selected_indices = []
    removed = set()

    # 각 기사에 대해 유사한 기사가 있으면 제거 (greedy 방식)
    for i in range(len(articles)):
        if i in removed:
            continue
        selected_indices.append(i)
        for j in range(i + 1, len(articles)):
            if sim_matrix[i][j] >= similarity_threshold:
                removed.add(j)

    # 선택된 인덱스에 해당하는 기사만 반환
    filtered_articles = [articles[i] for i in selected_indices]
    return filtered_articles

# 사용 예시:
filtered_articles = filter_similar_articles(all_articles, similarity_threshold=0.8)
print(f"원본 기사 수: {len(all_articles)} -> 필터링 후: {len(filtered_articles)}")


원본 기사 수: 1303 -> 필터링 후: 991


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 감성 분석 모델 및 토크나이저 로드
model_name = "alsgyu/sentiment-analysis-fine-tuned-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 감성 분석 수행 함수
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return scores[0][2].item()  # 긍정 점수 (2에 해당하는 확률값)

# 뉴스 데이터에 감성 분석 수행
for item in filtered_articles:
    text = item.get("title","") + " " + item.get("description","")
    item["positive_score"] = analyze_sentiment(text)

# 긍정 점수 기준 상위 20개 뉴스 정렬
top_positive_news = sorted(filtered_articles, key=lambda x: x["positive_score"], reverse=True)[:50]

# 데이터프레임 생성 및 출력
df_top_positive = pd.DataFrame(top_positive_news)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/722k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [5]:
df_top_positive

,title,description,pubDate,link,positive_score
0,"스피츠모바일, 알뜰폰 시장 진출…가성비 전략 펼쳐",샤오미 네이버 공식 브랜드스토어 스피츠가 설립한 스피츠모바일이 혁신적 혜택으로 국내...,2025-02-26 09:00:00,https://n.news.naver.com/mnews/article/005/000...,0.977239
1,"알뜰폰 티플러스, AI '스노우'와 제휴 요금제 출시",알뜰폰 브랜드 티플러스가 AI 카메라 '스노우'(SNOW)와 제휴 요금제를 선보인다...,2025-03-11 15:40:00,https://n.news.naver.com/mnews/article/092/000...,0.975312
2,'빅블러 시대' KB국민은행 금융·통신 융합 전략은,'MVNO 최초 5G 서비스' 및 '워치 요금제'를 도입해 새로운 시장을 개척했으며...,2025-03-17 19:00:00,https://www.startuptoday.co.kr/news/articleVie...,0.974782
3,"과기정통부, '알뜰폰' 시장 넓힌다...20GB·1만원대 요금도",정부는 아울러 이동통신사처럼 이용자 맞춤형 요금제를 자유롭게 출시할 수 있는 풀MV...,2025-01-15 11:48:00,http://www.ftoday.co.kr/news/articleView.html?...,0.972032
4,"알뜰폰 1위 프리티, 개인 맞춤 서비스와 글로벌 진출로 경쟁력 높인다",코스피 상장사 인스코비의 자회사인 알뜰폰 통신사인 프리텔레콤(프리티)이 고객 맞춤형...,2025-02-21 09:02:00,http://www.segyebiz.com/newsView/2025021950804...,0.970028
5,"프리티, 개인 맞춤 서비스와 글로벌 진출로 경쟁력 확보…&quot;알뜰폰 1위 굳...",내 알뜰폰 통신사인 '프리텔레콤(프리티)'이 고객 맞춤형 서비스와 글로벌 진출 전략...,2025-02-21 08:52:00,http://www.bizwnews.com/news/articleView.html?...,0.968892
6,KB국민 이어 우리도 참전… 은행 알뜰폰 영토 경쟁,내달 '우리WON 모바일' 오픈 경쟁 강화로 중소 사업자 숨통 고객 유치·비금융 수...,2025-03-06 16:00:00,https://n.news.naver.com/mnews/article/029/000...,0.967488
7,"KB국민은행, KB리브모바일 통해 미래 경쟁력 확보","또한 오프라인 서비스센터를 보유하지 않은 중소 MVNO 회원사를 위해 공동마케팅, ...",2025-03-17 07:30:00,http://www.hansbiz.co.kr/news/articleView.html...,0.967152
8,LGU+ “알뜰폰 불편 해결 쉬워진다”,온라인고객센터 ‘알닷케어’ 오픈 알뜰폰 파트너사 CS 업무 담당 가입 정보·요금 조...,2025-01-06 11:27:00,https://n.news.naver.com/mnews/article/016/000...,0.964987
9,20기가 1만원대 알뜰폰 요금제 출시,알뜰폰 시장에 1만원대 5세대(5G) 이동통신 20기가(GB) 요금제가 새롭게 출시...,2025-03-04 18:36:00,https://www.safetimes.co.kr/news/articleView.h...,0.963750


In [6]:
df_top_positive.to_csv('/content/drive/MyDrive/sideproject/naver_news/positive_news_top.csv', index=False, encoding="utf-8-sig")

In [9]:
top_positive_news_300 = sorted(filtered_articles, key=lambda x: x["positive_score"], reverse=True)[:300]

df_top_positive_300 = pd.DataFrame(top_positive_news_300)


In [10]:
df_top_positive_300.to_csv('/content/drive/MyDrive/sideproject/naver_news/positive_news_top_300.csv', index=False, encoding="utf-8-sig")